In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [ ]:
!pip install tensorflow==2.3.0
!pip install tensorflow-addons==0.13.0
!pip install numpy==1.19.0

## Load from Drive